<a href="https://colab.research.google.com/github/guikingoak/Estudos_Big_Data_Analytics/blob/main/ExDepartamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

**Questão 1**

Carregue os arquivos departments.csv, employees.csv e sales.csv em DataFrames Spark separados. Em seguida, exiba o schema e as primeiras linhas de cada.

In [ ]:
spark = SparkSession.builder.appName("Carregar Csv").getOrCreate()


departments_df = spark.read.csv('departments.csv', header=True, inferSchema=True)
employees_df = spark.read.csv('employees.csv', header=True, inferSchema=True)
sales_df = spark.read.csv('sales.csv', header=True, inferSchema=True)

departments_df.printSchema()
departments_df.show()

employees_df.printSchema()
employees_df.show()

sales_df.printSchema()
sales_df.show()

root
 |-- dept_id: integer (nullable = true)
 |-- dept_name: string (nullable = true)

+-------+-----------+
|dept_id|  dept_name|
+-------+-----------+
|      1|      Sales|
|      2|         HR|
|      3|Engineering|
+-------+-----------+

root
 |-- emp_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dept_id: integer (nullable = true)
 |-- salary: integer (nullable = true)

+------+-------+---+-------+------+
|emp_id|   name|age|dept_id|salary|
+------+-------+---+-------+------+
|   101|    Ana| 30|      1|  3000|
|   102|   Beto| 45|      1|  4000|
|   103| Carlos| 25|      2|  2000|
|   104|  Diana| 40|      3|  5000|
|   105|Ernesto| 35|      3|  4500|
+------+-------+---+-------+------+

root
 |-- emp_id: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- sales_amount: integer (nullable = true)

+------+-------+------------+
|emp_id|  month|sales_amount|
+------+-------+------------+
|   101|2021-01|    

**Questão 2**

Utilizando o DataFrame de funcionários (employees) e o DataFrame de departamentos (departments), realize um join para criar um novo DataFrame que contenha as colunas: emp_id, name, age, salary, dept_name. Em seguida, exiba as primeiras linhas deste novo DataFrame.

In [ ]:
employee_department_df = employees_df.join(departments_df, employees_df.dept_id == departments_df.dept_id, "inner") \
    .select(employees_df.emp_id, employees_df.name, employees_df.age, employees_df.salary, departments_df.dept_name)

employee_department_df.show()

+------+-------+---+------+-----------+
|emp_id|   name|age|salary|  dept_name|
+------+-------+---+------+-----------+
|   101|    Ana| 30|  3000|      Sales|
|   102|   Beto| 45|  4000|      Sales|
|   103| Carlos| 25|  2000|         HR|
|   104|  Diana| 40|  5000|Engineering|
|   105|Ernesto| 35|  4500|Engineering|
+------+-------+---+------+-----------+



**Questão 3**

A partir do DataFrame combinado da questão anterior, filtre para obter apenas os funcionários com mais de 30 anos de idade que pertencem ao departamento "Engineering". Exiba o resultado.

In [ ]:
filtrado_df = employee_department_df.filter((employee_department_df.age > 30) & (employee_department_df.dept_name == "Engineering"))
filtrado_df.show()

+------+-------+---+------+-----------+
|emp_id|   name|age|salary|  dept_name|
+------+-------+---+------+-----------+
|   104|  Diana| 40|  5000|Engineering|
|   105|Ernesto| 35|  4500|Engineering|
+------+-------+---+------+-----------+



**Questão 4**

Usando o DataFrame de vendas (sales), calcule o total de sales_amount por funcionário (emp_id). Exiba o resultado ordenando pela soma do valor de vendas, do maior para o menor, e mostre apenas os 2 funcionários com maior total de vendas.

In [ ]:
from pyspark.sql.functions import sum, desc

sales_by_employee = sales_df.groupBy("emp_id").agg(sum("sales_amount").alias("total_sales_amount"))

sorted_sales = sales_by_employee.orderBy(desc("total_sales_amount"))

sorted_sales.show(2)

+------+------------------+
|emp_id|total_sales_amount|
+------+------------------+
|   104|             25000|
|   101|             12000|
+------+------------------+
only showing top 2 rows



**Questão 5**

Salve o resultado da questão anterior (os top 2 vendedores) em um arquivo CSV.

In [ ]:
import pandas as pd

top_2_vend_df = sorted_sales.limit(2).toPandas()
top_2_vend_df.to_csv("top_2_vend.csv", index=False)